In [80]:
# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

# Set random seed for reproducibility
import random
random.seed(42)
import numpy as np
np.random.seed(42)

# Mushroom Classification

## Dataset (mushrooms.csv)
- Size: 8124 instances
- Features: 22 categorical features in the original dataset
- Target: class (edible or poisonous)

### Analysis Uses Simplified Feature Set:
9 selected features (simplified to at most 3 categories each):
1. cap-color: gray, brown, other
2. odor: foul, none, other
3. stalk-surface-above-ring: smooth, silky, other
4. stalk-surface-below-ring: smooth, silky, other
5. stalk-color-above-ring: pink, white, other
6. stalk-color-below-ring: pink, white, other
7. ring-type: pendant, evanescent, other
8. population: several, other
9. habitat: wood, grass, other

In [81]:
# Load the mushroom dataset
mushroom_table = MyPyTable()
mushroom_table.load_from_file('mushrooms.csv')

print(f"Dataset loaded: {len(mushroom_table.data)} instances")
print(f"Columns: {mushroom_table.column_names}")

Dataset loaded: 8124 instances
Columns: ['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']


In [82]:
# Helper function to run 10-fold cross-validation and display results
def evaluate_feature_subset(X, y, subset_name, feature_names):
    """Run 10-fold cross-validation and compute metrics for feature subset."""
    print("\n" + "="*80)
    print(f"{subset_name}")
    print(f"Features: {', '.join(feature_names)}")
    print("="*80)
    
    # Perform 10-fold cross-validation
    folds = myevaluation.kfold_split(X, n_splits=10, random_state=42, shuffle=True)
    
    all_y_true = []
    all_y_pred = []
    
    for train_indices, test_indices in folds:
        # Split data
        X_train = [X[i] for i in train_indices]
        y_train = [y[i] for i in train_indices]
        X_test = [X[i] for i in test_indices]
        y_test = [y[i] for i in test_indices]
        
        # Train and predict
        tree = MyDecisionTreeClassifier()
        tree.fit(X_train, y_train)
        y_pred = tree.predict(X_test)
        
        all_y_true.extend(y_test)
        all_y_pred.extend(y_pred)
    
    # Compute metrics
    accuracy = myevaluation.accuracy_score(all_y_true, all_y_pred)
    error_rate = 1 - accuracy
    
   
    labels = sorted(list(set(all_y_true)))
    print(f"Accuracy:    {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Error Rate:  {error_rate:.4f} ({error_rate*100:.2f}%)")
    
    # Per class metrics
    print(f"\nPer-Class Performance Metrics:")
    print(f"{'Class':<15} {'Precision':>12} {'Recall':>12} {'F1-Score':>12}")
    print("-" * 53)
    
    for label in labels:
        precision = myevaluation.binary_precision_score(all_y_true, all_y_pred, label)
        recall = myevaluation.binary_recall_score(all_y_true, all_y_pred, label)
        f1 = myevaluation.binary_f1_score(all_y_true, all_y_pred, label)
        print(f"{label:<15} {precision:>12.4f} {recall:>12.4f} {f1:>12.4f}")
    
    # Confusion
    conf_matrix = myevaluation.confusion_matrix(all_y_true, all_y_pred, labels)
    myutils.print_confusion_matrix(conf_matrix, labels, "\nConfusion Matrix")
    
    return accuracy, tree

## Step 1: Using Only the Odor Feature

This is a baseline set of results showing how accurately the model can predict whether a mushroom is edible or poisonous using only the odor feature.

In [83]:
odor_idx = mushroom_table.column_names.index('odor')
label_idx = mushroom_table.column_names.index('class')

X_odor = [[row[odor_idx]] for row in mushroom_table.data]
y = [row[label_idx] for row in mushroom_table.data]

acc_odor, tree_odor = evaluate_feature_subset(X_odor, y, "Odor Only", ['odor'])


Odor Only
Features: odor
Accuracy:    0.9852 (98.52%)
Error Rate:  0.0148 (1.48%)

Per-Class Performance Metrics:
Class              Precision       Recall     F1-Score
-----------------------------------------------------
e                     0.9723       1.0000       0.9859
p                     1.0000       0.9694       0.9844


Confusion Matrix:
----------------------------------------
                        e         p | Total
----------------------------------------
              e      4208         0 | 4208
              p       120      3796 | 3916
----------------------------------------
          Total      4328      3796 | 8124


## Step 2: Multiple Feature Subsets

Testing 4 different feature subsets to find the best combination for predicting mushroom edibility.

In [84]:
# feature subsets
feature_subsets = [
    {
        'name': 'Subset 1: Odor and Cap Color',
        'features': ['odor', 'cap-color']
    },
    {
        'name': 'Subset 2: Odor and Stalk Features',
        'features': ['odor', 'stalk-surface-above-ring', 'stalk-color-above-ring']
    },
    {
        'name': 'Subset 3: Odor and Ring and Habitat',
        'features': ['odor', 'habitat']
    },
    {
        'name': 'Subset 4: Comprehensive (5 features)',
        'features': ['odor', 'cap-color', 'stalk-color-below-ring', 'population', 'habitat']
    }
]

# Store results to compare
results = [{'name': 'Odor Only', 'accuracy': acc_odor, 'features': ['odor']}]

for subset in feature_subsets:
    # Get feature indices
    feature_indices = [mushroom_table.column_names.index(f) for f in subset['features']]
    
    # get features
    X_subset = [[row[idx] for idx in feature_indices] for row in mushroom_table.data]
    
    # Evaluate
    acc, tree = evaluate_feature_subset(X_subset, y, subset['name'], subset['features'])
    results.append({'name': subset['name'], 'accuracy': acc, 'features': subset['features'], 'tree': tree})


Subset 1: Odor and Cap Color
Features: odor, cap-color
Accuracy:    0.9882 (98.82%)
Error Rate:  0.0118 (1.18%)

Per-Class Performance Metrics:
Class              Precision       Recall     F1-Score
-----------------------------------------------------
e                     0.9777       1.0000       0.9887
p                     1.0000       0.9755       0.9876


Confusion Matrix:
----------------------------------------
                        e         p | Total
----------------------------------------
              e      4208         0 | 4208
              p        96      3820 | 3916
----------------------------------------
          Total      4304      3820 | 8124

Subset 2: Odor and Stalk Features
Features: odor, stalk-surface-above-ring, stalk-color-above-ring
Accuracy:    0.9862 (98.62%)
Error Rate:  0.0138 (1.38%)

Per-Class Performance Metrics:
Class              Precision       Recall     F1-Score
-----------------------------------------------------
e                     

In [85]:
# Summary comparison
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print()
print(f"{'Subset':<45} {'# Features':>10} {'Accuracy':>12} {'Error Rate':>12}")
print("-"*80)

for result in results:
    error_rate = 1 - result['accuracy']
    print(f"{result['name']:<45} {len(result['features']):>10} "
          f"{result['accuracy']:>12.4f} {error_rate:>12.4f}")

# Find best subset
best_result = max(results, key=lambda x: x['accuracy'])
print("\n" + "="*80)
print("BEST SUBSET")
print("="*80)
print(f"Name:      {best_result['name']}")
print(f"Features:  {', '.join(best_result['features'])}")
print(f"Accuracy:  {best_result['accuracy']:.4f} ({best_result['accuracy']*100:.2f}%)")
print(f"Error:     {(1-best_result['accuracy']):.4f} ({(1-best_result['accuracy'])*100:.2f}%)")
print("="*80)

# Show improvement over baseline
baseline_acc = results[0]['accuracy']
improvement = best_result['accuracy'] - baseline_acc
if improvement > 0:
    print(f"\nImprovement over baseline (Odor Only): +{improvement:.4f} "
          f"({improvement*100:.2f} percentage points)")
elif improvement < 0:
    print(f"\nChange from baseline (Odor Only): {improvement:.4f} "
          f"({improvement*100:.2f} percentage points)")
else:
    print(f"\nPerformance equals baseline (Odor Only)")



SUMMARY

Subset                                        # Features     Accuracy   Error Rate
--------------------------------------------------------------------------------
Odor Only                                              1       0.9852       0.0148
Subset 1: Odor and Cap Color                           2       0.9882       0.0118
Subset 2: Odor and Stalk Features                      3       0.9862       0.0138
Subset 3: Odor and Ring and Habitat                    2       0.9897       0.0103
Subset 4: Comprehensive (5 features)                   5       1.0000       0.0000

BEST SUBSET
Name:      Subset 4: Comprehensive (5 features)
Features:  odor, cap-color, stalk-color-below-ring, population, habitat
Accuracy:  1.0000 (100.00%)
Error:     0.0000 (0.00%)

Improvement over baseline (Odor Only): +0.0148 (1.48 percentage points)


## Decision Rules from Best Model

Training the best feature subset on the entire dataset to extract decision rules.

In [86]:
# Train best model on entire dataset
print("\n" + "="*80)
print("DECISION RULES FROM BEST MODEL")
print("="*80)
print(f"\nFeatures used: {best_result['features']}\n")

# Get feature indices for best subset
best_feature_indices = [mushroom_table.column_names.index(f) for f in best_result['features']]
X_best = [[row[idx] for idx in best_feature_indices] for row in mushroom_table.data]

# Train on entire dataset
final_tree = MyDecisionTreeClassifier()
final_tree.fit(X_best, y)

# Print decision rules
final_tree.print_decision_rules(attribute_names=best_result['features'], class_name='label')


DECISION RULES FROM BEST MODEL

Features used: ['odor', 'cap-color', 'stalk-color-below-ring', 'population', 'habitat']

IF odor == a THEN label = e
IF odor == c THEN label = p
IF odor == f THEN label = p
IF odor == l THEN label = e
IF odor == m THEN label = p
IF odor == n AND cap-color == b AND population == a THEN label = e
IF odor == n AND cap-color == b AND population == c THEN label = e
IF odor == n AND cap-color == b AND population == n THEN label = e
IF odor == n AND cap-color == b AND population == s THEN label = e
IF odor == n AND cap-color == b AND population == v THEN label = p
IF odor == n AND cap-color == b AND population == y THEN label = e
IF odor == n AND cap-color == c THEN label = e
IF odor == n AND cap-color == e THEN label = e
IF odor == n AND cap-color == g THEN label = e
IF odor == n AND cap-color == n AND stalk-color-below-ring == b THEN label = e
IF odor == n AND cap-color == n AND stalk-color-below-ring == c THEN label = e
IF odor == n AND cap-color == n AND s

## Analysis and Pruning Discussion

In [87]:
print("\n" + "="*80)
print("ANALYSIS")
print("="*80)

print("""
- Rules with very few instances are not very applicable and can be pruned and replaced with majority class
- Multiple rules leading to the same class with similar conditions can be merged them
- For each internal node, if removing that subtree and replacing with majority class doesn't affect accuracy on validation set it can be pruned
- Leaf nodes with low confidence can be replace with parent's majority or mark as uncertain
- The most important features can be kept and rules that split on less important features in the tree can be pruned
- Pruning decisions should be based on: validation set results, instance counts at each node, classification confidence for leaf nodes
""")


ANALYSIS

- Rules with very few instances are not very applicable and can be pruned and replaced with majority class
- Multiple rules leading to the same class with similar conditions can be merged them
- For each internal node, if removing that subtree and replacing with majority class doesn't affect accuracy on validation set it can be pruned
- Leaf nodes with low confidence can be replace with parent's majority or mark as uncertain
- The most important features can be kept and rules that split on less important features in the tree can be pruned
- Pruning decisions should be based on: validation set results, instance counts at each node, classification confidence for leaf nodes



## Conclusions
- The odor feature alone provides strong and reliable baseline performance.
- Adding more features beyond odor may provide marginal improvements, but risks overfitting on the training data.
- The subset with the highest accuracy balances predictive power with model simplicity.

- Pruning the decision tree would:
   - Reduce overfitting
   - Improve generalization 
   - Create more interpretable rules
   - Keep high accuracy with reduced complexity

In [88]:
def report_change(original_accuracy, modified_accuracy, 
                  original_leaves, modified_leaves,
                  original_stats, modified_stats):
    """
    Report the improvement/difference before and after optimization.
    
    Args:
        original_accuracy: Accuracy before optimization
        modified_accuracy: Accuracy after optimization
        original_leaves: Number of leaves before optimization
        modified_leaves: Number of leaves after optimization
        original_stats: Statistics dict for original tree
        modified_stats: Statistics dict for modified tree
    """
    
    # Tree complexity comparison
    print("\nTREE COMPLEXITY")
    print("-" * 80)
    print(f"{'Metric':<30} {'Original':>15} {'Modified':>15} {'Change':>15}")
    print("-" * 80)
    
    leaves_reduction = original_leaves - modified_leaves
    leaves_pct = (leaves_reduction / original_leaves * 100) if original_leaves > 0 else 0
    print(f"{'Number of Leaf Nodes':<30} {original_leaves:>15} {modified_leaves:>15} "
          f"{-leaves_reduction:>15} ({-leaves_pct:.1f}%)")
    
    orig_avg = sum(original_stats['counts']) / len(original_stats['counts'])
    modified_avg = sum(modified_stats['counts']) / len(modified_stats['counts'])
    print(f"{'Avg Instances per Leaf':<30} {orig_avg:>15.1f} {modified_avg:>15.1f} "
          f"{modified_avg - orig_avg:>15.1f}")

    # Performance comparison
    print("\nPREDICTIVE PERFORMANCE")
    print("-" * 80)
    print(f"{'Metric':<30} {'Original':>15} {'Modified':>15} {'Change':>15}")
    print("-" * 80)
    
    orig_error = 1 - original_accuracy
    modified_error = 1 - modified_accuracy
    accuracy_diff = modified_accuracy - original_accuracy
    error_diff = modified_error - orig_error
    
    print(f"{'Accuracy':<30} {original_accuracy:>14.4f} {modified_accuracy:>14.4f} "
          f"{accuracy_diff:>+14.4f}")
    print(f"{'Error Rate':<30} {orig_error:>14.4f} {modified_error:>14.4f} "
          f"{error_diff:>+14.4f}")
    
    print("="*80)

## Pruning by Removing Rules with Few Instances

Implementing pruning to remove rules with very few training instances and replace them with majority class predictions.

In [89]:
import copy

def prune_tree(tree, min_instances=10):
    """Prune decision tree by removing leaves with few instances.
    
    Args:
        tree: The decision tree to prune (nested list format)
        min_instances: Minimum number of instances required to keep a leaf
        
    Returns:
        Pruned tree
    """
    if tree[0] == "Leaf":
        # If it's a leaf, return as-is (pruning happens at parent level)
        return tree
    
    if tree[0] == "Attribute":
        # Process each value branch
        pruned_tree = ["Attribute", tree[1]]
        
        # Get all leaves from children to determine if should prune
        leaves = []
        for i in range(2, len(tree)):
            value_branch = tree[i]  # ["Value", value, subtree]
            if value_branch[0] == "Value":
                value = value_branch[1]
                subtree = value_branch[2]
                
                # Recursively prune subtree
                pruned_subtree = prune_tree(subtree, min_instances)
                
                # Get leaf information
                if pruned_subtree[0] == "Leaf":
                    leaves.append({
                        'label': pruned_subtree[1],
                        'count': pruned_subtree[2],
                        'parent_count': pruned_subtree[3]
                    })
                
                pruned_tree.append(["Value", value, pruned_subtree])
        
        # Check if all children are leaves with low instance counts
        if leaves and len(leaves) == len(tree) - 2:  # -2 for "Attribute" and attribute name
            total_instances = sum(leaf['count'] for leaf in leaves)
            
            # If instances below threshold, collapse to one leaf
            if total_instances < min_instances:
                # Find majority class of the leaves
                label_counts = {}
                for leaf in leaves:
                    label = leaf['label']
                    label_counts[label] = label_counts.get(label, 0) + leaf['count']
                
                majority_label = max(label_counts.items(), key=lambda x: x[1])[0]
                
                # Return collapsed leaf (use first leaf's parent_count)
                return ["Leaf", majority_label, total_instances, leaves[0]['parent_count']]
        
        return pruned_tree
    
    return tree


def count_leaves(tree):
    """Count the number of leaf nodes in a tree."""
    if tree[0] == "Leaf":
        return 1
    elif tree[0] == "Attribute":
        count = 0
        for i in range(2, len(tree)):
            value_branch = tree[i]
            if value_branch[0] == "Value":
                count += count_leaves(value_branch[2])
        return count
    return 0


def get_leaf_statistics(tree, stats=None):
    """Get statistics about leaf nodes."""
    if stats is None:
        stats = {'counts': [], 'labels': []}
    
    if tree[0] == "Leaf":
        stats['counts'].append(tree[2])
        stats['labels'].append(tree[1])
    elif tree[0] == "Attribute":
        for i in range(2, len(tree)):
            value_branch = tree[i]
            if value_branch[0] == "Value":
                get_leaf_statistics(value_branch[2], stats)
    
    return stats

original_tree = copy.deepcopy(best_result['tree'].tree)
original_leaves = count_leaves(original_tree)
original_stats = get_leaf_statistics(original_tree)

# Apply pruning with min_instances threshold
MIN_INSTANCES = 15
print(f"\nPruning with min_instances = {MIN_INSTANCES}")
pruned_tree_obj = copy.deepcopy(best_result['tree'])
pruned_tree_obj.tree = prune_tree(original_tree, min_instances=MIN_INSTANCES)
pruned_leaves = count_leaves(pruned_tree_obj.tree)
pruned_stats = get_leaf_statistics(pruned_tree_obj.tree)

# Evaluate pruned tree
print(f"\n{'='*80}")
print("PRUNED TREE EVALUATION")
print(f"{'='*80}")

# Evaluate on same data using cross-validation
folds = myevaluation.kfold_split(X_best, n_splits=10, random_state=42, shuffle=True)

all_y_true = []
all_y_pred = []

for train_indices, test_indices in folds:
    X_train_fold = [X_best[i] for i in train_indices]
    y_train_fold = [y[i] for i in train_indices]
    X_test_fold = [X_best[i] for i in test_indices]
    y_test_fold = [y[i] for i in test_indices]
    
    # Train and prune
    tree = MyDecisionTreeClassifier()
    tree.fit(X_train_fold, y_train_fold)
    tree.tree = prune_tree(tree.tree, min_instances=MIN_INSTANCES)
    
    # Predict
    y_pred = tree.predict(X_test_fold)
    
    all_y_true.extend(y_test_fold)
    all_y_pred.extend(y_pred)

# Compute metrics
pruned_accuracy = myevaluation.accuracy_score(all_y_true, all_y_pred)

# Call the improvement report function
report_change(
    best_result['accuracy'], 
    pruned_accuracy,
    original_leaves,
    pruned_leaves,
    original_stats,
    pruned_stats
)

# Print pruned decision rules
print(f"\n{'='*80}")
print("NEW DECISION RULES")
print(f"{'='*80}\n")
pruned_tree_obj.print_decision_rules(attribute_names=best_result['features'], class_name='class')


Pruning with min_instances = 15

PRUNED TREE EVALUATION

TREE COMPLEXITY
--------------------------------------------------------------------------------
Metric                                Original        Modified          Change
--------------------------------------------------------------------------------
Number of Leaf Nodes                        59              59               0 (-0.0%)
Avg Instances per Leaf                   123.9           123.9             0.0

PREDICTIVE PERFORMANCE
--------------------------------------------------------------------------------
Metric                                Original        Modified          Change
--------------------------------------------------------------------------------
Accuracy                               1.0000         0.9990        -0.0010
Error Rate                             0.0000         0.0010        +0.0010

NEW DECISION RULES

IF odor == a THEN class = e
IF odor == c THEN class = p
IF odor == f THEN class =

## Rule Merging: Combining Similar Rules with Same Outcome

Implementing rule merging to simplify the tree by combining multiple branches that lead to the same classification.

In [122]:
def merge_similar_rules(tree, merge_threshold=0.8):
    """
    Merge rules that lead to the same class with similar conditions.
    
    If a majority (>= merge_threshold) of branches from an attribute node lead to the same class,
    collapse that node to a single leaf with that class.
    
    Args:
        tree: The decision tree to simplify (nested list format)
        merge_threshold: Fraction of branches that must agree (0.0 to 1.0)
                        Default 0.8 means 80% of branches must lead to same class
    
    Returns:
        Simplified tree with merged rules
    """
    if tree[0] == "Leaf":
        return tree
    
    if tree[0] == "Attribute":
        # First, recursively process all children
        merged_tree = ["Attribute", tree[1]]
        child_leaves = []
        
        for i in range(2, len(tree)):
            value_branch = tree[i]
            if value_branch[0] == "Value":
                value = value_branch[1]
                subtree = value_branch[2]
                
                # Recursively merge the subtree
                merged_subtree = merge_similar_rules(subtree, merge_threshold)
                merged_tree.append(["Value", value, merged_subtree])
                
                # Collect leaf information
                if merged_subtree[0] == "Leaf":
                    child_leaves.append({
                        'label': merged_subtree[1],
                        'count': merged_subtree[2],
                        'parent_count': merged_subtree[3]
                    })
        
        # Check if we can merge based on class agreement
        if child_leaves and len(child_leaves) == len(tree) - 2:  # All children are leaves
            # Count instances by label
            label_counts = {}
            total_instances = 0
            
            for leaf in child_leaves:
                label = leaf['label']
                count = leaf['count']
                label_counts[label] = label_counts.get(label, 0) + count
                total_instances += count
            
            # Find the majority class
            if label_counts:
                majority_label = max(label_counts.items(), key=lambda x: x[1])[0]
                majority_count = label_counts[majority_label]
                
                # Check if majority exceeds threshold
                if total_instances > 0:
                    majority_fraction = majority_count / total_instances
                    
                    if majority_fraction >= merge_threshold:
                        # Merge into a single leaf
                        return ["Leaf", majority_label, total_instances, child_leaves[0]['parent_count']]
        
        return merged_tree
    
    return tree


def count_rules(tree, path=[]):
    """Count the number of decision rules (leaf nodes) in a tree."""
    if tree[0] == "Leaf":
        return 1
    elif tree[0] == "Attribute":
        total = 0
        for i in range(2, len(tree)):
            value_branch = tree[i]
            if value_branch[0] == "Value":
                total += count_rules(value_branch[2])
        return total
    return 0

original_tree_copy = copy.deepcopy(best_result['tree'].tree)

MERGE_THRESHOLD = 0.8
print(f"\nMerge threshold: {MERGE_THRESHOLD*100:.0f}% (branches must agree this much to merge)")

# Create merged tree
merged_tree_obj = copy.deepcopy(best_result['tree'])
merged_tree_obj.tree = merge_similar_rules(original_tree_copy, merge_threshold=MERGE_THRESHOLD)

# Count rules before and after
original_rules = count_rules(original_tree_copy)
merged_rules = count_rules(merged_tree_obj.tree)
merged_leaves = count_leaves(merged_tree_obj.tree)
merged_stats = get_leaf_statistics(merged_tree_obj.tree)

print(f"\nOriginal Tree:")
print(f"  Decision rules: {original_rules}")
print(f"  Leaf nodes: {original_leaves}")

print(f"\nMerged Tree:")
print(f"  Decision rules: {merged_rules} (reduced by {original_rules - merged_rules})")
print(f"  Leaf nodes: {merged_leaves} (reduced by {original_leaves - merged_leaves})")
if merged_stats['counts']:
    print(f"  Avg instances per leaf: {sum(merged_stats['counts'])/len(merged_stats['counts']):.1f}")

# Evaluate merged tree
print(f"\n{'='*80}")
print("MERGED TREE EVALUATION")
print(f"{'='*80}")

folds = myevaluation.kfold_split(X_best, n_splits=10, random_state=42, shuffle=True)

all_y_true = []
all_y_pred = []

for train_indices, test_indices in folds:
    X_train_fold = [X_best[i] for i in train_indices]
    y_train_fold = [y[i] for i in train_indices]
    X_test_fold = [X_best[i] for i in test_indices]
    y_test_fold = [y[i] for i in test_indices]
    
    # Train and merge
    tree = MyDecisionTreeClassifier()
    tree.fit(X_train_fold, y_train_fold)
    tree.tree = merge_similar_rules(tree.tree, merge_threshold=MERGE_THRESHOLD)
    
    # Predict
    y_pred = tree.predict(X_test_fold)
    
    all_y_true.extend(y_test_fold)
    all_y_pred.extend(y_pred)

# Compute metrics
merged_accuracy = myevaluation.accuracy_score(all_y_true, all_y_pred)

# Use the reporting function
report_change(
    best_result['accuracy'], 
    merged_accuracy,
    original_leaves,
    merged_leaves,
    original_stats,
    merged_stats
)

# Print decision rules
print(f"\n{'='*80}")
print("NEW DECISION RULES")
print(f"{'='*80}\n")
final_tree.print_decision_rules(attribute_names=best_result['features'], class_name='class')


Merge threshold: 80% (branches must agree this much to merge)

Original Tree:
  Decision rules: 59
  Leaf nodes: 59

Merged Tree:
  Decision rules: 59 (reduced by 0)
  Leaf nodes: 59 (reduced by 0)
  Avg instances per leaf: 123.9

MERGED TREE EVALUATION

TREE COMPLEXITY
--------------------------------------------------------------------------------
Metric                                Original        Modified          Change
--------------------------------------------------------------------------------
Number of Leaf Nodes                        59              59               0 (-0.0%)
Avg Instances per Leaf                   123.9           123.9             0.0

PREDICTIVE PERFORMANCE
--------------------------------------------------------------------------------
Metric                                Original        Modified          Change
--------------------------------------------------------------------------------
Accuracy                               1.0000         0.

## Validation-Based Pruning

Testing each internal node by seeing if replacing the subtree with majority class doesn't hurt validation accuracy and pruning in this case.

In [121]:
def validation_prune_tree(tree, X_val, y_val, tree_obj):
    """
    Prune tree using validation set - replace subtrees with majority class if accuracy doesn't decrease.
    
    This is reduced error pruning: for each internal node, test if replacing it with a leaf
    improves or maintains validation accuracy.
    
    Args:
        tree: The decision tree structure (nested list)
        X_val: Validation features
        y_val: Validation labels
        tree_obj: The tree object (for prediction)
    
    Returns:
        Pruned tree
    """
    # Get baseline accuracy with current tree
    baseline_predictions = tree_obj.predict(X_val)
    baseline_accuracy = myevaluation.accuracy_score(y_val, baseline_predictions)
    
    # Try pruning this node
    pruned_tree = _validation_prune_recursive(tree, X_val, y_val, tree_obj, baseline_accuracy)
    
    return pruned_tree


def _validation_prune_recursive(tree, X_val, y_val, tree_obj, baseline_accuracy):
    """
    Recursively test pruning each subtree against validation set.
    """
    if tree[0] == "Leaf":
        return tree
    
    if tree[0] == "Attribute":
        # First, recursively prune children
        pruned_tree = ["Attribute", tree[1]]
        
        for i in range(2, len(tree)):
            value_branch = tree[i]
            if value_branch[0] == "Value":
                value = value_branch[1]
                subtree = value_branch[2]
                
                # Recursively prune the subtree
                pruned_subtree = _validation_prune_recursive(subtree, X_val, y_val, tree_obj, baseline_accuracy)
                pruned_tree.append(["Value", value, pruned_subtree])
        
        # Now test if we should prune this entire node
        # Collect all leaf labels under this node
        leaf_labels = []
        _collect_leaf_labels(pruned_tree, leaf_labels)
        
        if leaf_labels:
            # Find majority class
            label_counts = {}
            for label in leaf_labels:
                label_counts[label] = label_counts.get(label, 0) + 1
            majority_label = max(label_counts.items(), key=lambda x: x[1])[0]
            
            # Create a version with this node replaced by a leaf
            test_tree = ["Leaf", majority_label, sum(label_counts.values()), sum(label_counts.values())]
            
            # Test accuracy with pruned version
            original_tree = tree_obj.tree
            tree_obj.tree = test_tree
            pruned_predictions = tree_obj.predict(X_val)
            pruned_accuracy = myevaluation.accuracy_score(y_val, pruned_predictions)
            tree_obj.tree = original_tree  # Restore
            
            # If accuracy doesn't decrease (or improves), prune it
            if pruned_accuracy >= baseline_accuracy:
                return test_tree
        
        return pruned_tree
    
    return tree


def _collect_leaf_labels(tree, labels):
    """Collect all leaf labels from a tree."""
    if tree[0] == "Leaf":
        labels.append(tree[1])
    elif tree[0] == "Attribute":
        for i in range(2, len(tree)):
            value_branch = tree[i]
            if value_branch[0] == "Value":
                _collect_leaf_labels(value_branch[2], labels)


print("="*80)
print("VALIDATION-BASED PRUNING")
print("="*80)

# Use k-fold cross-validation
folds = myevaluation.kfold_split(X_best, n_splits=10, random_state=42, shuffle=True)

all_y_true = []
all_y_pred = []
prune_counts = []

for fold_idx, (train_indices, val_indices) in enumerate(folds):
    # Split into train and validation
    X_train_fold = [X_best[i] for i in train_indices]
    y_train_fold = [y[i] for i in train_indices]
    X_val_fold = [X_best[i] for i in val_indices]
    y_val_fold = [y[i] for i in val_indices]
    
    # Train tree
    tree = MyDecisionTreeClassifier()
    tree.fit(X_train_fold, y_train_fold)
    
    # Count leaves before pruning
    leaves_before = count_leaves(tree.tree)
    
    # Apply validation-based pruning
    tree.tree = validation_prune_tree(tree.tree, X_val_fold, y_val_fold, tree)
    
    # Count leaves after pruning
    leaves_after = count_leaves(tree.tree)
    prune_counts.append(leaves_before - leaves_after)
    
    # Predict on validation set
    y_pred = tree.predict(X_val_fold)
    
    all_y_true.extend(y_val_fold)
    all_y_pred.extend(y_pred)

# Compute overall metrics
validation_pruned_accuracy = myevaluation.accuracy_score(all_y_true, all_y_pred)

# Get statistics about a single validation-pruned tree for comparison
final_tree = MyDecisionTreeClassifier()
final_tree.fit(X_best, y)

# Split for validation (use 10% as validation)
from random import sample
val_size = len(X_best) // 10
val_indices = sample(range(len(X_best)), val_size)
train_indices = [i for i in range(len(X_best)) if i not in val_indices]

X_train_final = [X_best[i] for i in train_indices]
y_train_final = [y[i] for i in train_indices]
X_val_final = [X_best[i] for i in val_indices]
y_val_final = [y[i] for i in val_indices]

final_tree.fit(X_train_final, y_train_final)
original_leaves_final = count_leaves(final_tree.tree)
final_tree.tree = validation_prune_tree(final_tree.tree, X_val_final, y_val_final, final_tree)
validation_pruned_leaves = count_leaves(final_tree.tree)
validation_pruned_stats = get_leaf_statistics(final_tree.tree)

report_change(
    best_result['accuracy'],
    validation_pruned_accuracy,
    original_leaves,
    validation_pruned_leaves,
    original_stats,
    validation_pruned_stats
)

# Print decision rules
print(f"\n{'='*80}")
print("NEW DECISION RULES")
print(f"{'='*80}\n")
final_tree.print_decision_rules(attribute_names=best_result['features'], class_name='class')


VALIDATION-BASED PRUNING

TREE COMPLEXITY
--------------------------------------------------------------------------------
Metric                                Original        Modified          Change
--------------------------------------------------------------------------------
Number of Leaf Nodes                        59              59               0 (-0.0%)
Avg Instances per Leaf                   123.9           123.9             0.0

PREDICTIVE PERFORMANCE
--------------------------------------------------------------------------------
Metric                                Original        Modified          Change
--------------------------------------------------------------------------------
Accuracy                               1.0000         1.0000        +0.0000
Error Rate                             0.0000         0.0000        +0.0000

NEW DECISION RULES

IF odor == a THEN class = e
IF odor == c THEN class = p
IF odor == f THEN class = p
IF odor == l THEN class = e
I

In [127]:
def optimize_tree(tree, tree_obj, X_val=None, y_val=None, min_instances=15, merge_threshold=0.8, use_validation_pruning=True):
    """
    Apply validation-based pruning, instance-based pruning, and rule merging for optimal tree simplification.
    
    Args:
        tree: The decision tree to optimize
        tree_obj: The tree classifier object (needed for validation pruning)
        X_val: Validation features (optional, needed for validation pruning)
        y_val: Validation labels (optional, needed for validation pruning)
        min_instances: Minimum instances for instance-based pruning
        merge_threshold: Agreement threshold for merging
        use_validation_pruning: Whether to apply validation-based pruning
    
    Returns:
        Optimized tree
    """
    current_tree = tree
    
    # Validation-based pruning
    current_tree = validation_prune_tree(current_tree, X_val, y_val, tree_obj)
    
    # Apply instance-based pruning (remove low-instance rules)
    current_tree = prune_tree(current_tree, min_instances=min_instances)
    
    # Apply rule merging 
    current_tree = merge_similar_rules(current_tree, merge_threshold=merge_threshold)
    
    return current_tree


print("="*80)
print("COMBINED OPTIMIZATION")
print("="*80)

# Use cross-validation to evaluate the full optimization pipeline
folds = myevaluation.kfold_split(X_best, n_splits=10, random_state=42, shuffle=True)

all_y_true = []
all_y_pred = []

for train_indices, test_indices in folds:
    X_train_fold = [X_best[i] for i in train_indices]
    y_train_fold = [y[i] for i in train_indices]
    X_test_fold = [X_best[i] for i in test_indices]
    y_test_fold = [y[i] for i in test_indices]
    
    # Further split training into train/validation for validation pruning
    val_size = len(X_train_fold) // 10
    val_indices_inner = list(range(len(X_train_fold)))[:val_size]
    train_indices_inner = list(range(len(X_train_fold)))[val_size:]
    
    X_train_inner = [X_train_fold[i] for i in train_indices_inner]
    y_train_inner = [y_train_fold[i] for i in train_indices_inner]
    X_val_inner = [X_train_fold[i] for i in val_indices_inner]
    y_val_inner = [y_train_fold[i] for i in val_indices_inner]
    
    # Train tree on inner training set
    tree = MyDecisionTreeClassifier()
    tree.fit(X_train_inner, y_train_inner)
    
    # Apply full optimization pipeline with validation data
    tree.tree = optimize_tree(
        tree.tree, 
        tree,
        X_val=X_val_inner,
        y_val=y_val_inner,
        min_instances=MIN_INSTANCES, 
        merge_threshold=MERGE_THRESHOLD,
        use_validation_pruning=True
    )
    
    # Predict on test set
    y_pred = tree.predict(X_test_fold)
    
    all_y_true.extend(y_test_fold)
    all_y_pred.extend(y_pred)

# Compute metrics
full_optimized_accuracy = myevaluation.accuracy_score(all_y_true, all_y_pred)

# Create a single optimized tree for display
from random import sample
val_size = len(X_best) // 10
val_indices = sample(range(len(X_best)), val_size)
train_indices = [i for i in range(len(X_best)) if i not in val_indices]

X_train_display = [X_best[i] for i in train_indices]
y_train_display = [y[i] for i in train_indices]
X_val_display = [X_best[i] for i in val_indices]
y_val_display = [y[i] for i in val_indices]

display_tree = MyDecisionTreeClassifier()
display_tree.fit(X_train_display, y_train_display)
original_leaves_display = count_leaves(display_tree.tree)
original_stats_display = get_leaf_statistics(display_tree.tree)

display_tree.tree = optimize_tree(
    display_tree.tree,
    display_tree,
    X_val=X_val_display,
    y_val=y_val_display,
    min_instances=MIN_INSTANCES,
    merge_threshold=MERGE_THRESHOLD,
    use_validation_pruning=True
)

full_optimized_leaves = count_leaves(display_tree.tree)
full_optimized_stats = get_leaf_statistics(display_tree.tree)
full_optimized_rules = count_rules(display_tree.tree)

print(f"\nTree Complexity:")
print(f"  Original:   {original_leaves_display} leaves")
print(f"  Optimized:  {full_optimized_leaves} leaves")
print(f"  Reduction:  {original_leaves_display - full_optimized_leaves} leaves ({(original_leaves_display - full_optimized_leaves)/original_leaves_display*100:.1f}%)")

report_change(
    best_result['accuracy'], 
    full_optimized_accuracy,
    original_leaves,
    full_optimized_leaves,
    original_stats,
    full_optimized_stats
)

# Print optimized decision rules
print(f"\n{'='*80}")
print("NEW DECISION RULES")
print(f"{'='*80}\n")
display_tree.print_decision_rules(attribute_names=best_result['features'], class_name='class')

# Summary comparison of all techniques
print(f"{'\nTechnique':<35} {'Leaves':>10} {'Accuracy':>12} ")
print("-"*75)
print(f"{'Original Tree':<35} {original_leaves:>10} {best_result['accuracy']:>12.4f} ")
print(f"{'Instance-based Pruning':<35} {pruned_leaves:>10} {pruned_accuracy:>12.4f} ")
print(f"{'Rule Merging':<35} {merged_leaves:>10} {merged_accuracy:>12.4f} ")
print(f"{'Validation-based Pruning':<35} {validation_pruned_leaves:>10} {validation_pruned_accuracy:>12.4f} ")
print(f"{'Full Pipeline':<35} {full_optimized_leaves:>10} {full_optimized_accuracy:>12.4f} ")
print("-"*75)

COMBINED OPTIMIZATION

Tree Complexity:
  Original:   59 leaves
  Optimized:  40 leaves
  Reduction:  19 leaves (32.2%)

TREE COMPLEXITY
--------------------------------------------------------------------------------
Metric                                Original        Modified          Change
--------------------------------------------------------------------------------
Number of Leaf Nodes                        59              40             -19 (-32.2%)
Avg Instances per Leaf                   123.9           182.8            58.9

PREDICTIVE PERFORMANCE
--------------------------------------------------------------------------------
Metric                                Original        Modified          Change
--------------------------------------------------------------------------------
Accuracy                               1.0000         0.9972        -0.0028
Error Rate                             0.0000         0.0028        +0.0028

NEW DECISION RULES

IF odor == a THEN